In [1]:
!pip install openai yfinance pandas pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=296950736c04c5732c92010cbfdb097f26a79248f647a5cc459e0181de6132d7
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta


In [2]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from datetime import datetime

In [3]:
# 내 보유 주식 객체
class MyStock:
    def __init__(self, symbol, name, quantity, price, date):
        self.symbol = symbol
        self.name = name
        self.quantity = quantity   # 보유 수량
        self.price = price         # 매수 단가
        self.date = date           # 매수 일자

# 보유 주식 리스트
my_stocks = [
    MyStock("AAPL", "Apple", 10, 224.2, "2024-11-11"),
    MyStock("MSFT", "Microsoft", 5, 417.17, "2024-11-11"),
    MyStock("NVDA", "NVIDIA", 3, 145.24, "2024-11-11"),
    MyStock("GOOGL", "Alphabet", 2, 180.14, "2024-11-11"),
    MyStock("META", "Meta Platforms", 1, 582.69, "2024-11-11")
]

# 관심 종목을 리스트로 보관
stocks = [
    ("AAPL", "Apple"),
    ("MSFT", "Microsoft"),
    ("NVDA", "NVIDIA"),
    ("GOOGL", "Alphabet"),
    ("META", "Meta Platforms"),
    ("AMZN", "Amazon"),
    ("AMD", "Advanced Micro Devices"),
    ("TSLA", "Tesla"),
    ("JNJ", "Johnson & Johnson"),
    ("JPM", "JPMorgan Chase"),
    ("BAC", "Bank of America"),
    ("XOM", "Exxon Mobil"),
    ("NEE", "NextEra Energy"),
    ("BA", "Boeing"),
    ("UNP", "Union Pacific"),
    ("KO", "Coca-Cola"),
    ("COST", "Costco"),
    ("DIS", "Disney"),
]

In [4]:

def generate_technical_report(stocks, my_stocks, period: str = "2y", interval: str = "1d"):
    """
    주어진 종목(stocks)에 대해 start~end까지의 데이터를 다운로드하고,
    RSI, MACD, Bollinger Bands 지표를 계산한 뒤,
    내가 보유 중인 종목(my_stocks)에 대해서는 추가 매도 평가 정보를 함께 기재
    """
    # 보유 주식을 symbol 기준으로 빠르게 찾기 위해 dict 변환
    my_stocks_dict = {s.symbol: s for s in my_stocks}

    reports = {}

    for ticker, name in stocks:
        df = yf.download(ticker, period=period, interval=interval, progress=False)

        if df.empty:
            reports[ticker] = f"{ticker} ({name}) 데이터가 존재하지 않습니다."
            continue

        # 데이터 정제
        df = df.dropna()
        # 멀티 인덱스인 경우 제거
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.droplevel(1)

        # RSI
        df["RSI_14"] = ta.rsi(df["Close"], length=14)

        # MACD
        macd = ta.macd(df["Close"], fast=12, slow=26, signal=9)
        df["MACD"] = macd["MACD_12_26_9"]
        df["MACD_SIGNAL"] = macd["MACDs_12_26_9"]
        df["MACD_HIST"] = macd["MACDh_12_26_9"]

        # Bollinger Bands
        bb = ta.bbands(df["Close"], length=20, std=2)
        df["BB_UPPER"] = bb["BBU_20_2.0"]
        df["BB_MIDDLE"] = bb["BBM_20_2.0"]
        df["BB_LOWER"] = bb["BBL_20_2.0"]

        # 결과 추출
        last_date = df.index[-1]
        current_data = df.iloc[-1]

        current_close = current_data["Close"]
        rsi_14 = current_data["RSI_14"]
        macd_val = current_data["MACD"]
        macd_signal = current_data["MACD_SIGNAL"]
        macd_hist = current_data["MACD_HIST"]
        bb_upper = current_data["BB_UPPER"]
        bb_middle = current_data["BB_MIDDLE"]
        bb_lower = current_data["BB_LOWER"]

        # 최근 5영업일 종가 (예: tail(5))
        recent_closes = df["Close"].tail(5)

        # 최근 종가 히스토리 문자열 생성
        # (예) "2025-01-10: 300.21, 2025-01-11: 305.12, ..."
        recent_closes_str = ", ".join([f"{idx.date()}: {val:.2f}" for idx, val in recent_closes.items()])

        # 보고서 기본 정보
        report = f"""
======== {name} ({ticker}) ========
일자: {last_date.date()}
종가: {current_close:.2f}

[최근 5영업일 종가]
{recent_closes_str}

[RSI(14)]
{rsi_14:.2f}

[MACD(12,26,9)]
MACD: {macd_val:.2f}
Signal: {macd_signal:.2f}
Hist: {macd_hist:.2f}

[Bollinger Bands(20,2)]
상단선(Upper): {bb_upper:.2f}
중간선(Middle): {bb_middle:.2f}
하단선(Lower): {bb_lower:.2f}
"""

        # 만약 해당 종목을 보유 중이라면, 매도 평가 관련 내용 추가
        if ticker in my_stocks_dict:
            my_info = my_stocks_dict[ticker]

            # 보유 정보
            quantity = my_info.quantity
            buy_price = my_info.price   # 매수 단가
            buy_date = my_info.date

            # 손익 계산 임시로 했음, TODO 라이브러리로 대체
            unrealized_pnl = (current_close - buy_price) * quantity
            pnl_rate = ((current_close - buy_price) / buy_price) * 100

            sell_evaluation = f"""
[보유 정보]
매수 일자: {buy_date}
보유 수량: {quantity}주
매수 단가: {buy_price:.2f}

[손익]
미실현 손익: {unrealized_pnl:,.2f} USD
수익률: {pnl_rate:.2f}%
"""
            report += sell_evaluation

        report += "============================="

        reports[ticker] = report.strip()

    return reports

# 전체 보고서 생성
full_report_dict = generate_technical_report(stocks, my_stocks)
# 확인용 보고서 출력
print(full_report_dict["AAPL"])

======== Apple (AAPL) ========
일자: 2025-01-16
종가: 228.65

[최근 5영업일 종가]
2025-01-10: 236.85, 2025-01-13: 234.40, 2025-01-14: 233.28, 2025-01-15: 237.87, 2025-01-16: 228.65

[RSI(14)]
32.88

[MACD(12,26,9)]
MACD: -2.49
Signal: 0.16
Hist: -2.65

[Bollinger Bands(20,2)]
상단선(Upper): 263.47
중간선(Middle): 246.23
하단선(Lower): 229.00

[보유 정보]
매수 일자: 2024-11-11
보유 수량: 10주
매수 단가: 224.20

[손익]
미실현 손익: 44.45 USD
수익률: 1.98%


In [5]:

def summarize_report_for_prompt(full_report_dict):
    """
    기존 full_report_dict를 핵심 요약만 추출해 OpenAI에 전달할 문자열을 생성

    Return: prompt_text (str)
    """
    prompt_lines = []

    for ticker, full_text in full_report_dict.items():
        # 비용 절약용으로 종목명, 종가, RSI, MACD, 보유시 손익률 등을 정규표현식이나 단순 파싱으로 추출
        # 근데 input 비용 적어서 크게 의미 있나 싶음 TODO 나중에 비용 체크하기

        lines = full_text.splitlines()

        # 일단 바로 파싱 TODO: 정규식으로 뽑기
        name_line = [l for l in lines if ticker in l]
        current_price_line = [l for l in lines if "종가:" in l]
        rsi_line = [l for l in lines if "[RSI(14)]" in l or "RSI(14)" in l]
        macd_line = [l for l in lines if "[MACD(12,26,9)]" in l or "MACD:" in l]
        buy_line = [l for l in lines if "매수 단가:" in l]
        pnl_line = [l for l in lines if "수익률:" in l]

        # 원하는 문자열만 조합
        summary = []
        if name_line:
            summary.append(name_line[0].replace("======== ", "").replace(" ========", ""))
        if buy_line:
            summary.append("보유")
        else:
            summary.append("관심")
        if current_price_line:
            summary.append(current_price_line[0].strip())
        if rsi_line:
            # RSI 값 한 줄 아래에 있을 수도 있으니, 해당 인덱스+1 라인까지 추출
            idx = lines.index(rsi_line[0])
            # 예시: RSI(14)이 있는 줄 바로 다음에 실제 RSI 값
            if idx+1 < len(lines):
                summary.append(lines[idx+1].strip())
        if macd_line:
            # MACD 라인이 여러 줄일 수 있으므로 조금만 수집
            macd_idx = lines.index(macd_line[0])
            # 예시로 MACD: ~ / Signal: ~ / Hist: ~ 3줄 정도 추출
            for i in range(1,4):
                if macd_idx+i < len(lines):
                    summary.append(lines[macd_idx+i].strip())

        if buy_line:
            # 매수 항목
            summary.append(buy_line[0].strip())

        if pnl_line:
            # 수익률 항목
            summary.append(pnl_line[0].strip())

        if summary:
            prompt_lines.append(" / ".join(summary))

    # 최종 프롬프트용 문자열
    prompt_text = "\n".join(prompt_lines)
    return prompt_text

# 토큰 비용 비교용으로 보고서 요약본 생성
short_report = summarize_report_for_prompt(full_report_dict)

print(short_report)

Apple (AAPL) / 보유 / 종가: 228.65 / 32.88 / MACD: -2.49 / Signal: 0.16 / Hist: -2.65 / 매수 단가: 224.20 / 수익률: 1.98%
Microsoft (MSFT) / 보유 / 종가: 426.63 / 48.96 / MACD: -3.07 / Signal: -2.39 / Hist: -0.68 / 매수 단가: 417.17 / 수익률: 2.27%
NVIDIA (NVDA) / 보유 / 종가: 134.89 / 46.67 / MACD: -0.70 / Signal: -0.21 / Hist: -0.49 / 매수 단가: 145.24 / 수익률: -7.13%
Alphabet (GOOGL) / 보유 / 종가: 193.79 / 56.65 / MACD: 2.94 / Signal: 3.63 / Hist: -0.69 / 매수 단가: 180.14 / 수익률: 7.58%
Meta Platforms (META) / 보유 / 종가: 613.62 / 53.53 / MACD: 3.98 / Signal: 4.17 / Hist: -0.18 / 매수 단가: 582.69 / 수익률: 5.31%
Amazon (AMZN) / 관심 / 종가: 220.87 / 50.58 / MACD: 0.95 / Signal: 2.03 / Hist: -1.08
Advanced Micro Devices (AMD) / 관심 / 종가: 119.01 / 40.92 / MACD: -3.90 / Signal: -4.03 / Hist: 0.13
Tesla (TSLA) / 관심 / 종가: 410.94 / 52.42 / MACD: 6.36 / Signal: 10.53 / Hist: -4.17
Johnson & Johnson (JNJ) / 관심 / 종가: 147.51 / 54.00 / MACD: -1.18 / Signal: -1.79 / Hist: 0.61
JPMorgan Chase (JPM) / 관심 / 종가: 252.85 / 67.81 / MACD: 2.77 / Signal: 1

In [7]:

from google.colab import userdata
from openai import OpenAI # GPT-4 API


def create_prompt_text(report_text):
    prompt_intro = """
    아래는 내가 보유한 주식 및 관심 종목들에 대한 최근 기술적 지표와 손익(수익률) 요약 정보입니다.
    RSI, MACD, Bollinger Bands 등 핵심 정보를 바탕으로,

    각 종목에 대해
    보유 종목은 매수/매도/홀딩 중 어떤 의사결정을 고려할 수 있을지,
    관심 종목은 매수/홀딩 중 어떤 의사결정을 고려할 수 있을지,
    그리고 그 이유(간단히)를 제시해 주세요.
    """

    prompt_warning = """
    주의: 이 분석은 단순 참고용이며 실제 투자 결정은 책임질 수 없음을 압니다.
    """

    return f"{prompt_intro}\n\n{report_text}\n\n{prompt_warning}"


client = OpenAI(api_key=userdata.get('openapi_key'))

# 프롬프트 텍스트 생성
full_prompt_text = create_prompt_text("\n".join(full_report_dict.values()))
short_prompt_text = create_prompt_text(short_report)

full_response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "당신은 유능한 금융 애널리스트입니다. 사용자에게 투자 조언을 제공해주세요."},
        {"role": "user", "content": full_prompt_text}
    ],
    max_tokens=4096, # 충분한 응답 보증
    temperature=0.5
)

full_assistant_reply = full_response.choices[0].message.content
print(full_assistant_reply)

short_response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "당신은 유능한 금융 애널리스트입니다. 사용자에게 투자 조언을 제공해주세요."},
        {"role": "user", "content": short_prompt_text}
    ],
    max_tokens=4096,
    temperature=0.5
)

short_assistant_reply = short_response.choices[0].message.content
print(short_assistant_reply)


### 보유 종목

#### Apple (AAPL)
- **의사결정:** 홀딩
- **이유:** RSI가 32.88로 과매도 영역에 근접해 있어 추가 하락의 가능성이 있지만, 현재 가격이 볼린저 밴드 하단선에 근접하여 반등 가능성도 존재합니다. 미실현 손익이 플러스 상태이므로, 반등을 기다리며 홀딩하는 것이 좋습니다.

#### Microsoft (MSFT)
- **의사결정:** 홀딩
- **이유:** RSI가 48.96으로 중립 영역에 있으며, MACD 히스토그램이 약간 부정적이지만 큰 변동을 보이지 않습니다. 미실현 손익이 플러스 상태이므로, 현재 상태를 유지하며 추이를 지켜보는 것이 좋습니다.

#### NVIDIA (NVDA)
- **의사결정:** 매도 고려
- **이유:** RSI가 46.67로 중립에 가깝지만 하락세에 있으며, MACD 역시 부정적입니다. 미실현 손익이 마이너스 상태로, 손실을 줄이기 위해 매도를 고려할 수 있습니다.

#### Alphabet (GOOGL)
- **의사결정:** 홀딩
- **이유:** RSI가 56.65로 중립에서 약간 강세이며, 미실현 손익이 플러스 상태입니다. 기술적 지표가 크게 악화되지 않았으므로 홀딩할 수 있습니다.

#### Meta Platforms (META)
- **의사결정:** 홀딩
- **이유:** RSI가 53.53으로 중립에 있으며, 미실현 손익이 플러스입니다. 기술적 지표가 안정적이므로 현재 상태를 유지하는 것이 좋습니다.

### 관심 종목

#### Amazon (AMZN)
- **의사결정:** 홀딩
- **이유:** RSI가 50.58로 중립이며, MACD 히스토그램이 부정적이지만 큰 변동은 없습니다. 중립적인 상태를 유지하며 추이를 지켜볼 수 있습니다.

#### Advanced Micro Devices (AMD)
- **의사결정:** 매수 고려
- **이유:** RSI가 40.92로 과매도에 가까우며, MACD 히스토그램이 약간 긍정적으로 전환 중입니다. 저점 매수를 고려할 수 있습니

TODO
 - 기술적 지표 외 차트 분석 방식 추가 고려
 - RAG를 활용한 실시간 데이터 추가
  * 현재 고정적인 데이터 공급 라인을 탐색중
  * 실시간 정보가 거래에 미치는 가중치 계산 필요
 - 재무제표 분석 기능 추가
  * 마찬가지로 거래에 미치는 가중치 계산 필요